In [14]:
import pandas as pd
import numpy as np 
from numpy.linalg import svd

In [25]:
store = pd.read_csv('listcoffee.csv',index_col=0).drop_duplicates()
rating = pd.read_csv('listrating.csv',index_col=0).drop_duplicates(['User_ID','Store_ID'])

In [13]:
store.head()

,Store,type,Store_ID
0,NAP's Coffee & Roasters,speedbar,0
178,sangob,speedbar,1
235,SongSarn,slowbar,2
287,LIFE Roasters,"hybrid,bakery",3
542,Nap x Warin,speedbar,4


In [27]:
rating

,User_ID,Store_ID,Rating
0,33,0,5
1,34,0,3
2,35,0,5
3,36,0,5
4,37,0,5
...,...,...,...
3348,2266,32,5
3349,2267,32,5
3350,2268,32,5
3351,1831,32,5


In [29]:
rating.pivot(index='User_ID',columns='Store_ID',values='Rating').fillna(0)

Store_ID,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User_ID,,,,,,,,,,,,,,,,,,,,,
33,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,3.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,5.0,0.0,5.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,5.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
37,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [30]:
u, sigma, v = svd(rating.values)

In [37]:
u.shape

(3285, 3285)